## basic_linear_regression: 
A simple single layer pytorch model to show how ```torch.nn.Linear``` solves general multi-variate linear regression problems

___
### Idea:
The pytorch nn.Linear class can solve general Linear Equations of the kind:  $$\mathbf{y} = \mathbf{x}\mathbf{A^\top} + b$$ where:
* $\mathbf{A}$ is a transformation matrix which represent coefficients of the equation, 
* $\mathbf{x}$ is a vector of inputs and 
* $\mathbf{y}$ is a vector that is the result of the linear transformation

We are given: 
* rows of $\mathbf{x}$ values in a matrix $\mathbf{X}$,
* rows of $\mathbf{y}$ values in a matrix $\mathbf{Y}$

The model <span style="color:blue">SingleLayerNet</span> solves for the transformation matrix $\mathbf{A}$.

___
### Use:
##### In section 1.0
* set the scalar variable ```number_of_coefficients```
 * this variable determines the number of columns in Matrix $\mathbf{A}$
* set the scalar variable ```y_dimension```
 * the y_dimension determines:
   * the size of the $\mathbf{y}$ vector output of the linear transformation
   * this variable determines the number of rows in Matrix $\mathbf{A}$
* set the scalar bias term
* a scalar, (e.g. 0.1) in order to add some normally distributed random noise to the linear transformation

```
# exzmple from section 1.0
number_of_coefficients=3
y_dimension=number_of_coefficients
bias = 1
noise_level = 0.1
```

##### In section 2.0:
* run all cells from 2.0 on 

##### In section 3.0:
* run a single $\mathbf{x}$ vector test to see if the model's coefficients work
___

___
## 1.0 - Set the values below to determine: 
* the shape of matrix $\mathbf{A}$
* the shape of the output vector $\mathbf{y}$
* the amount of bias (the b term of the above linear equation)

In [1]:
number_of_coefficients=3
y_dimension=number_of_coefficients
bias = 1
noise_level = 0.1

___
## 2.0  - Run all cells below to determine the values in the matrix $\mathbf{A}$

#### 2.01 Imports

In [2]:
import pandas as pd
import numpy as np
import pypg.pg_pandas as pg
import os,sys
import pdb
import torch 
from torch import nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F


/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### 2.02 ```create_x_values``` creates a single row of training data

In [3]:
# this method builds test y values/vectors for training
def create_x_values(x_vector):
    m = number_of_coefficients
    A = np.linspace(1,m,m).repeat(m).reshape(-1,m)
    return np.matmul(A,x_vector) + bias + np.random.randn() * noise_level

#### 2.03 Define the SingleLayerNet model

In [4]:
# main model class
class SingleLayerNet(nn.Module):
  def __init__(self, D_in, D_out):
    super(SingleLayerNet, self).__init__()
    self.linear1 = nn.Linear(D_in, D_out) 
  def forward(self, x):
    return self.linear1(x)


In [5]:
# number of training rows
n=10000
# number of elements in each batch on each epoch
b=100
# number of epochs
epochs=7000
# instantiate model
m1 = SingleLayerNet(number_of_coefficients,y_dimension)
# Create input torch Variables for X and Y
Xnp = np.random.rand(n,number_of_coefficients)
X = Variable(torch.Tensor(Xnp))
Ynp = np.array([create_x_values(x) for x in X])
Y = Variable(torch.Tensor(Ynp).reshape(-1,number_of_coefficients))

# create loss and optimizer
loss_fn = torch.nn.MSELoss(size_average = False) 
optimizer = optim.Adam(m1.parameters(), lr = 0.01)

# Training loop
for i in range(epochs):
    # create a batch of x values and y values (labels)
    indices = list(range(n))
    np.random.shuffle(indices)
    xb = X[indices[:b]]    
    yb = Y[indices][:b]
    # zero the optimizer
    optimizer.zero_grad()  # clear previous gradients
    
    # execute the forward pass to compute y values from equation xA^T + b (the linear transformation)
    output_batch = m1(xb)           # compute model output
    
    # calculate a loss
    loss = loss_fn(output_batch, yb)  # calculate loss

    # compute gradients
    loss.backward()        # compute gradients of all variables wrt loss
    optimizer.step()       # perform updates using calculated gradients
    # print out progress
    if i % 500 == 0 :
        print('epoch {}, loss {}'.format(i,loss.data[0]))

# print model results
model_A = m1.linear1.weight.data.numpy()
model_bias = m1.linear1.bias.data.numpy()
print(f'A = {model_A}')
print(f'bias = {model_bias}')

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 6075.51123046875
epoch 500, loss 16.192283630371094
epoch 1000, loss 8.863194465637207
epoch 1500, loss 5.04934024810791
epoch 2000, loss 2.679178237915039
epoch 2500, loss 3.3506500720977783
epoch 3000, loss 2.822608470916748
epoch 3500, loss 4.0402326583862305
epoch 4000, loss 3.608442544937134
epoch 4500, loss 2.945143938064575
epoch 5000, loss 3.089378595352173
epoch 5500, loss 3.0989437103271484
epoch 6000, loss 3.0009024143218994
epoch 6500, loss 2.9494965076446533
A = [[1.0062019 1.0010154 0.9928246]
 [2.0036192 2.0022457 1.9948812]
 [3.0040717 3.0039475 2.9970388]]
bias = [0.9933811  0.99323434 0.9940275 ]


___
## 3.0 run a test on a single x vector

In [6]:
example_x = np.linspace(1,number_of_coefficients,number_of_coefficients)
print(f'example x value {example_x}')
example_y = create_x_values(example_x)
print(f'example y value to predict {example_y}')
example_prediction = m1(torch.Tensor(example_x.reshape(-1,number_of_coefficients))).data.numpy()
print(f'example y prediction {example_prediction}')
diffs = example_y - example_prediction
print(f'difference =  {diffs.round(4)}')

example x value [1. 2. 3.]
example y value to predict [ 7.1710942 13.1710942 19.1710942]
example y prediction [[ 6.9800878 12.985989  18.99711  ]]
difference =  [[0.191  0.1851 0.174 ]]


## End